In [1]:
from __future__ import print_function
import torch
from torch.autograd import Variable, Function
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

import scipy.io as sio
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from minimax_entropy import MinimaxEntropyEstimator

In [12]:
entro = MinimaxEntropyEstimator('poly_coeff_entro.mat', gpu=True)

# Training settings
batch_size = 64
test_batch_size = 1000
epochs = 1
random_seed = 1
log_interval = 10
dropout = 0.2
classes = 10
n_samples = 10

In [3]:
# loading data
torch.cuda.manual_seed(random_seed)

kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=False,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=test_batch_size, shuffle=True, **kwargs)

In [9]:
class Net(nn.Module):
    def __init__(self, dropout=0.):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d(dropout)
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
        
        self._dropout = dropout

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.dropout(F.relu(self.fc1(x)), p=self._dropout, training=self.training)
        x = F.softmax(self.fc2(x))
        return x

In [10]:
model = Net(dropout=dropout)
model.cuda()

optimizer = optim.Adam(model.parameters())

def train(epoch):
    model.train()
    
    target_onehot = Variable(torch.DoubleTensor(batch_size, 10)).cuda()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        
        target_onehot.data.zero_()
        for i in range(batch_size):
            target_onehot[i, target.data[i]] = 1.
        
        output = model(data).double()
        
        loss = Variable(torch.zeros(1)).double().cuda()
        for i in range(batch_size):
            loss += entro.cross_entro_loss(output[i], target_onehot[i])
        loss.backward()
        
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.data[0]))
        
        if batch_idx == 100:
            break
            
def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data[0] # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

for epoch in range(1, epochs + 1):
    train(epoch)
    test()

Train Epoch: 1 [0/60000 (0%)]	Loss: 207.153333
Train Epoch: 1 [640/60000 (1%)]	Loss: 199.270007
Train Epoch: 1 [1280/60000 (2%)]	Loss: 176.491838
Train Epoch: 1 [1920/60000 (3%)]	Loss: 149.817325
Train Epoch: 1 [2560/60000 (4%)]	Loss: 108.101410
Train Epoch: 1 [3200/60000 (5%)]	Loss: 90.329066
Train Epoch: 1 [3840/60000 (6%)]	Loss: 82.045769
Train Epoch: 1 [4480/60000 (7%)]	Loss: 69.678501
Train Epoch: 1 [5120/60000 (9%)]	Loss: 57.566432
Train Epoch: 1 [5760/60000 (10%)]	Loss: 60.702964
Train Epoch: 1 [6400/60000 (11%)]	Loss: 58.079881

Test set: Average loss: -0.8238, Accuracy: 8962/10000 (90%)

